In [3]:
import networkx as nx
import numpy as np
import os
import io
import json

In [16]:
# algorithm inspired by https://github.com/jeroenvldj/bow-tie_detection/blob/master/bow-tie_detection.py

def bowtie_analysis(G):
    # reverse all direction of the graph
    GT = nx.reverse(G, copy=True)
    # calculate SSC
    SSC = max(list(nx.strongly_connected_components(G)),key=len)    
    
    
    # take any node n from SSC and do a depth first search 
    # through directed graph beginning from node n
    v_any = list(SSC)[0]
    DFS_G = set(nx.dfs_tree(G,v_any).nodes())
    DFS_GT = set(nx.dfs_tree(GT,v_any).nodes())
    OUT = DFS_G - SSC
    IN = DFS_GT - SSC
    V_rest = set(G.nodes()) - SSC - OUT - IN

    TUBES = set()
    INTENDRILS = set()
    OUTTENDRILS = set()
    OTHER = set()

    for v in V_rest:
        # irv => in reaches node v
        irv = len(IN & set(nx.dfs_tree(GT,v).nodes())) is not 0
        # vro => node v reaches out
        vro = len(OUT & set(nx.dfs_tree(G,v).nodes())) is not 0
        if irv and vro:
            TUBES.add(v)
        elif irv and not vro:
            INTENDRILS.add(v)
        elif not irv and vro:
            OUTTENDRILS.add(v)
        elif not irv and not vro:
            OTHER.add(v)

    FRINGE = set()
    DISCONNECTED = set()
    for o in OTHER:
        # orIT => node o reaches INTENDRILS  
        orIT = len(INTENDRILS & set(nx.dfs_tree(G,o))) is not 0
        # OTro => OUTTERNDIRLS reaches node o
        OTro = len(OUTTENDRILS & set(nx.dfs_tree(GT,o))) is not 0
        if orIT or OTro:
            FRINGE.add(o)
        else:
            DISCONNECTED.add(o)
    
    TENDRILS = INTENDRILS.union(OUTTENDRILS)
    
    def component_result(name, graph_nodes):
        return{ name        : len(graph_nodes),
                name + "_s" : sum([G.nodes()[node]["is_sink"] for node in graph_nodes]),
                name + "_m" : sum([G.nodes()[node]["is_miner"] for node in graph_nodes])}

    result_dict = dict()
    result_dict.update(component_result("nodes", G.nodes()))
    result_dict.update(component_result("ssc", SSC))
    result_dict.update(component_result("in", IN))
    result_dict.update(component_result("out",OUT))
    result_dict.update(component_result("tubes", TUBES))
    result_dict.update(component_result("tendrils", TENDRILS))
    result_dict.update(component_result("fringe", FRINGE))
    result_dict.update(component_result("disconnected", DISCONNECTED))
    
    return result_dict

In [18]:
def files_walker(directory, json_output):
    d = {}
    files = os.listdir(directory)
    for file in files:
        with open(directory+'/'+file, 'r', encoding='utf8', errors='ignore') as f:
            G = nx.read_graphml(f)
            bowtie_dict = bowtie_analysis(G)
            with open(json_output, "r+") as fi:
                data = json.load(fi)
                new_entry = {file[:-8] : bowtie_dict}
                data.update(new_entry)
                fi.seek(0)
                json.dump(data, fi, indent=4)
    return True

In [19]:
# run bow tie analysis
json_output = "test.json"

if not os.path.isfile(json_output):
    with io.open(os.path.join(json_output), 'w') as db_file:
        db_file.write(json.dumps({}))
        
files_walker("data/NET-btc-heur_0-week", json_output)

True